In [1]:
from torch.utils.data import DataLoader 
import torch
from pyscf import gto, dft, scf 

from dpyscf.torch_routines import SCF
from dpyscf.net import XC, X_L, C_L
from dpyscf.utils import *


### 1. Define the xc model

Here we use a simple LDA model (`level = 1`), by choosing `heg_mult = True` (for homogeneous electron gas exchange) and `pw_mult = True` (for Perdew Wang HEG correlation), without any enhancement factors (`grid_models = []`). No Hartree-Fock exact exchange is used `exx_a=None`.

In [2]:
xc = XC(grid_models = [], heg_mult=True, pw_mult=True, level=1, exx_a=None)

Create the object that runs the SCF calculation

In [3]:
alpha_init = 0.8
alpha = torch.nn.Parameter(torch.Tensor([alpha_init]))
dscf = SCF(alpha=alpha, nsteps=10, xc=xc, device='cpu', exx=False)

### 2. Create a molecule with pyscf

This computes all overlap integrals and other information that will be used by our differentiable implementation

In [4]:
mol = gto.M(atom='H 0 0 0; H 0 0 1', basis='3-21G')
mf = scf.RKS(mol)
mf.grids.level = 1
mf.grids.build()
mf.kernel()
E_converged = mf.e_tot

converged SCF energy = -1.1074945503839


### 3. Create input data structures

In [5]:
# * The marked lines are just workarounds because we are only dealing with one system in this tutorial and both Dataset and DataLoader are designed 
# to deal with multiple data points 

matrices = get_datapoint(mol, mf)
matrices = {key: [val] for key,val in matrices.items()} # *
dset = Dataset(**matrices)
dloader = DataLoader(dset, batch_size = 1)
inputs = next(iter(dloader)) # *

### 4. Run the calculation

In [6]:
output = dscf.forward(*inputs)

In [7]:
output

{'E': tensor([-0.9104, -1.1270, -1.1086, -1.1076, -1.1075, -1.1074, -1.1074, -1.1074,
         -1.1074, -1.1074], grad_fn=<CatBackward>),
 'dm': tensor([[0.1309, 0.1871, 0.1309, 0.1871],
         [0.1871, 0.2675, 0.1871, 0.2675],
         [0.1309, 0.1871, 0.1309, 0.1871],
         [0.1871, 0.2675, 0.1871, 0.2675]], grad_fn=<PermuteBackward>),
 'mo_energy': tensor([-0.3316,  0.0164,  0.8137,  0.9951], grad_fn=<SymeigBackward>)}

## 5. Calculate the loss

In [8]:
E_converged_t = torch.Tensor([E_converged])

E_converged_t

tensor([-1.1075])

Mean squared error loss:

In [9]:
torch.sum((output['E'] - E_converged_t)**2)

tensor(0.0392, grad_fn=<SumBackward0>)

PyTorch-native MSE loss 

In [10]:
loss = torch.nn.MSELoss()
loss_value = loss(output['E'], E_converged_t.repeat(10))
loss_value

tensor(0.0039, grad_fn=<MseLossBackward>)

Calculate gradients with backprop

In [11]:
loss_value.backward()

In [12]:
alpha.grad

tensor([0.0008])

Optimizer

In [13]:
opt = torch.optim.SGD([alpha], lr=0.001)
opt.step()

In [14]:
print('Old alpha:', alpha_init)
print('New alpha:', alpha.detach().numpy()[0])

Old alpha: 0.8
New alpha: 0.7999992389941798
